In [1]:
import os
import rasterio
import numpy as np
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats


Raster Reclassification Process Documentation
Objective:
To reclassify multiple raster datasets representing various priority scores, scaling their values from their original ranges to a standardized 0-10 scale for improved readability and comparison.
Method:
A Python script utilizing the rasterio library was developed to automate the reclassification process. The script performs the following operations:

Iterates through all .tif files in a specified directory.
For each raster:
a. Calculates the minimum and maximum values, excluding NoData values.
b. Applies a linear rescaling formula to transform the original values to a 0-10 range:
new_value = (old_value - min_value) / (max_value - min_value) * 10
c. Preserves NoData values by assigning them a value of -9999 in the output.
d. Creates a new raster file with "_reclassified" appended to the original filename.
Outputs processing information for each raster, including original value ranges and confirmation of the new 0-10 range.

Implementation:
The script is executed on all raster files located in:
C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\priority
Output:

Reclassified rasters are saved in the same directory as the input files.
Each new raster maintains the spatial properties and coordinate system of the original.
Values in the new rasters range from 0 to 10, where 0 represents the lowest priority and 10 the highest priority from the original data.

Benefits:

Standardizes diverse datasets to a common scale.
Enhances interpretability for non-technical stakeholders.
Facilitates easier comparison between different priority metrics.
Automates the process, ensuring consistency and reducing manual error.

Note:
This reclassification maintains the relative relationships within each dataset but allows for standardized comparison across different priority metrics. Users should be aware that the absolute values have been transformed and should interpret the new 0-10 scale accordingly.

In [3]:

# Open the input raster
with rasterio.open(r"C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\eems_riparian_restoration_priority\riparian_restoration_priority.tif") as src:
    # Read the raster band as a numpy array
    data = src.read(1)
    
    # Get the current no data value
    nodata = src.nodata
    
    # Create a mask for valid data
    valid_mask = (data != nodata) & (data >= -1) & (data <= 0.83017998615461)
    
    # Create output array initialized with zeros (or choose another background value)
    output = np.zeros_like(data)
    
    # Reclassify valid data to 0-100 range
    output[valid_mask] = (data[valid_mask] - (-1)) / (0.83017998615461 - (-1)) * 100
    
    # Set invalid data to NoData (use -9999 or any other value you prefer)
    output[~valid_mask] = -9999
    
    # Prepare the metadata for the output raster
    profile = src.profile
    profile.update(dtype=rasterio.float32, nodata=-9999)

# Write the output raster
with rasterio.open(r"C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\eems_riparian_restoration_priority\riparian_restoration_priority_reclassified.tif", 'w', **profile) as dst:
    dst.write(output.astype(rasterio.float32), 1)

print("Reclassification complete!")

Reclassification complete!


In [22]:
def reclassify_raster(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1)
        nodata = src.nodata
        
        # Calculate min and max, ignoring nodata values
        valid_data = data[data != nodata]
        data_min = np.min(valid_data)
        data_max = np.max(valid_data)
        
        # Create mask for valid data
        valid_mask = (data != nodata) & (data >= data_min) & (data <= data_max)
        
        # Reclassify
        output = np.zeros_like(data)
        output[valid_mask] = (data[valid_mask] - data_min) / (data_max - data_min) * 10
        output[~valid_mask] = -9999
        
        profile = src.profile
        profile.update(dtype=rasterio.float32, nodata=-9999)

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(output.astype(rasterio.float32), 1)

    return data_min, data_max

In [23]:
# Folder containing the TIF files
raster_folder = r"C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\priority"

# Process each TIF file in the folder
for filename in os.listdir(raster_folder):
    if filename.endswith(".tif"):
        input_file = os.path.join(raster_folder, filename)
        output_file = os.path.join(raster_folder, f"{os.path.splitext(filename)[0]}_reclassified.tif")
        
        print(f"Processing: {filename}")
        min_val, max_val = reclassify_raster(input_file, output_file)
        print(f"Created: {os.path.basename(output_file)}")
        print(f"Original range: {min_val} to {max_val}")
        print("---")

print("All rasters have been reclassified!")

Processing: avocado_suitability.tif
Created: avocado_suitability_reclassified.tif
Original range: -1.0 to 0.9457849458876643
---
Processing: citrus_suitability.tif
Created: citrus_suitability_reclassified.tif
Original range: -1.0 to 0.9457849458876643
---
Processing: low_water_plantation_suitability.tif
Created: low_water_plantation_suitability_reclassified.tif
Original range: -1.0 to 0.9471396607018236
---
Processing: prescribed_herbivory.tif
Created: prescribed_herbivory_reclassified.tif
Original range: -1.0 to 0.8080095220963862
---
Processing: riparian_restoration_priority.tif
Created: riparian_restoration_priority_reclassified.tif
Original range: -1.0 to 0.8642051530435017
---
Processing: suitable_for_oak.tif
Created: suitable_for_oak_reclassified.tif
Original range: -1.0 to 0.808687962427745
---
Processing: vegetation_suitability.tif
Created: vegetation_suitability_reclassified.tif
Original range: -1.0 to 1.0
---
All rasters have been reclassified!


Zonal Statistics Calculation Process Documentation
Objective:
To calculate zonal statistics (mean and median) for multiple reclassified raster datasets using watershed boundaries, and compile the results into a single vector dataset for analysis and comparison.
Method:
A Python script utilizing geopandas, rasterio, and rasterstats libraries was developed to automate the zonal statistics calculation process. The script performs the following operations:

Reads a watershed boundary vector file (GeoPackage format) using geopandas.
Iterates through all reclassified raster files (identified by "_reclassified.tif" suffix) in a specified directory.
For each raster:
a. Calculates zonal statistics (mean and median) using the rasterstats library.
b. Rounds the calculated statistics to the nearest whole number.
c. Adds the results to the watershed GeoDataFrame as new attributes.
Saves the updated GeoDataFrame, including all original watershed data and new zonal statistics, as a new GeoPackage file.

Implementation:
The script processes all reclassified raster files located in:
C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\priority
It uses the watershed boundary file:
C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\priority\rwmp_watersheds_800ftsplit_firehzdscore_dslpwinds_epsg2229.gpkg
Output:

A new GeoPackage file named 'watersheds_with_stats.gpkg' is created in the same directory as the input rasters.
The output file contains all original watershed attributes plus new columns for each raster's mean and median values.
Column naming convention: [original_raster_name]_mean and [original_raster_name]_median

Benefits:

Automates the calculation of zonal statistics for multiple rasters.
Ensures consistency in calculations across all datasets.
Compiles results into a single, easily analyzable vector format.
Facilitates efficient comparison of different priority metrics across watersheds.

Note:
This process assumes that all raster datasets have been previously reclassified to a common scale (0-10 in this case). The resulting statistics represent the average and median priority scores for each watershed across various metrics.

In [24]:
def calculate_zonal_stats(raster_folder, vector_path):
    # Read the vector file
    gdf = gpd.read_file(vector_path)
    
    # Process each reclassified raster and calculate stats
    for filename in os.listdir(raster_folder):
        if filename.endswith("_reclassified.tif"):
            raster_path = os.path.join(raster_folder, filename)
            
            print(f"Calculating zonal statistics for: {filename}")
            
            # Calculate zonal statistics
            stats = zonal_stats(gdf, raster_path, stats=['median'])
            
            # Convert to DataFrame for easier handling
            stats_df = pd.DataFrame(stats)
            
            # Round  median to GeoDataFrame
            base_name = os.path.splitext(filename)[0].replace('_reclassified', '')
            gdf[f'{base_name}_median'] = stats_df['median'].round(0)
            
            print(f"Zonal statistics calculated for {filename}")

    # Save the updated GeoDataFrame
    output_vector = os.path.join(raster_folder, 'watersheds_with_stats.gpkg')
    gdf.to_file(output_vector, driver='GPKG')
    print(f"Updated vector file saved to: {output_vector}")

In [25]:
# Path to the watershed vector file
watershed_vector = r"C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\priority\rwmp_watersheds_800ftsplit_firehzdscore_dslpwinds_epsg2229.gpkg"


In [26]:
# Run the process
calculate_zonal_stats(raster_folder, watershed_vector)

print("Zonal statistics calculated for all reclassified rasters!")

Calculating zonal statistics for: avocado_suitability_reclassified.tif
Zonal statistics calculated for avocado_suitability_reclassified.tif
Calculating zonal statistics for: citrus_suitability_reclassified.tif
Zonal statistics calculated for citrus_suitability_reclassified.tif
Calculating zonal statistics for: low_water_plantation_suitability_reclassified.tif
Zonal statistics calculated for low_water_plantation_suitability_reclassified.tif
Calculating zonal statistics for: prescribed_herbivory_reclassified.tif
Zonal statistics calculated for prescribed_herbivory_reclassified.tif
Calculating zonal statistics for: riparian_restoration_priority_reclassified.tif
Zonal statistics calculated for riparian_restoration_priority_reclassified.tif
Calculating zonal statistics for: suitable_for_oak_reclassified.tif
Zonal statistics calculated for suitable_for_oak_reclassified.tif
Calculating zonal statistics for: vegetation_suitability_reclassified.tif
Zonal statistics calculated for vegetation_sui

In [44]:
# Path to the RWMP Project points file
project_points = r"C:\Users\bsf31\Documents\post-meds\data\signal\landscape\rwmp_project_sites.gpkg"

# Path to the raster files
raster_folder = r"C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\priority"

# Path to the fire hazard score vector file
hazard_score = r"C:\Users\bsf31\Documents\post-meds\data\signal\landscape\rwmp_watersheds_800ftsplit_firehzdscore_dslpwinds_epsg2229.gpkg"

# List of raster files
oak_suitability_raster = os.path.join(raster_folder, "suitable_for_oak_reclassified.tif")
rx_herbivory_raster = os.path.join(raster_folder, "prescribed_herbivory_reclassified.tif")
avo_raster = os.path.join(raster_folder, "avocado_suitability_reclassified.tif")
citrus_raster = os.path.join(raster_folder, "citrus_suitability_reclassified.tif")
low_water_raster = os.path.join(raster_folder, "low_water_plantation_suitability_reclassified.tif")
riparian_raster = os.path.join(raster_folder, "riparian_restoration_priority_reclassified.tif")


In [45]:
def buffer_and_extract(points, hazard_vector, rasters, buffer_size=100):
    gdf_points = gpd.read_file(points)
    gdf_points['geometry'] = gdf_points.geometry.buffer(buffer_size)
    gdf_hazard = gpd.read_file(hazard_vector)
    
    if gdf_points.crs != gdf_hazard.crs:
        gdf_hazard = gdf_hazard.to_crs(gdf_points.crs)
    
    results = []
    
    for index, row in gdf_points.iterrows():
        site_name = row['name']
        print(f"Processing site: {site_name}")
        
        site_results = {'Site Name': site_name}
        
        hazard_info = gdf_hazard[gdf_hazard.intersects(row['geometry'])]
        if not hazard_info.empty:
            site_results['Max Hazard Score'] = hazard_info['hzds__max'].values[0]
        else:
            site_results['Max Hazard Score'] = "No intersection"
        
        for raster_name, raster_path in rasters.items():
            print(f"  Processing raster: {raster_name}")
            stats = zonal_stats(row['geometry'], raster_path, stats=['median', 'count', 'min', 'max'])
            print(f"    Zonal stats result: {stats}")
            
            if stats[0]['count'] > 0:
                site_results[f'Med. {raster_name}'] = stats[0]['median']
                site_results[f'Min {raster_name}'] = stats[0]['min']
                site_results[f'Max {raster_name}'] = stats[0]['max']
            else:
                # If zonal stats fail, sample directly at the point
                with rasterio.open(raster_path) as src:
                    point_value = list(src.sample([row['geometry'].centroid.coords[0]]))[0][0]
                print(f"    Direct raster value at point: {point_value}")
                site_results[f'Med. {raster_name}'] = point_value
                site_results[f'Min {raster_name}'] = point_value
                site_results[f'Max {raster_name}'] = point_value
        
        results.append(site_results)
    
    return pd.DataFrame(results)

In [67]:
# Define rasters dictionary
rasters = {
    'Oak Suitability': oak_suitability_raster,
    'Rx Herbivory Suitability': rx_herbivory_raster,
    'Avo Suitability': avo_raster,
    'Citrus Suitability': citrus_raster,
    'Low-Water Suitability': low_water_raster,
    'Riparian Suitability': riparian_raster
}

# Run the analysis
result_df = buffer_and_extract(project_points, hazard_score, rasters)

# After creating the DataFrame, replace "No data" with NaN for consistency
result_df = result_df.replace("No data", pd.NA)

# Round only the numeric columns to whole numbers
numeric_columns = result_df.select_dtypes(include=[np.number]).columns
#result_df[numeric_columns] = result_df[numeric_columns].round(0)
result_df[numeric_columns] = result_df[numeric_columns].round(0)




Processing site: trout_club
  Processing raster: Oak Suitability
    Zonal stats result: [{'min': 5.1107563972473145, 'max': 5.1107563972473145, 'count': 1, 'median': 5.1107563972473145}]
  Processing raster: Rx Herbivory Suitability
    Zonal stats result: [{'min': 3.0714495182037354, 'max': 3.0714495182037354, 'count': 1, 'median': 3.0714495182037354}]
  Processing raster: Avo Suitability
    Zonal stats result: [{'min': 0.7927691340446472, 'max': 0.7927691340446472, 'count': 1, 'median': 0.7927691340446472}]
  Processing raster: Citrus Suitability
    Zonal stats result: [{'min': 0.7927691340446472, 'max': 0.7927691340446472, 'count': 1, 'median': 0.7927691340446472}]
  Processing raster: Low-Water Suitability
    Zonal stats result: [{'min': 0.7922175526618958, 'max': 0.7922175526618958, 'count': 1, 'median': 0.7922175526618958}]
  Processing raster: Riparian Suitability
    Zonal stats result: [{'min': 0.0, 'max': 0.0, 'count': 1, 'median': 0.0}]
Processing site: painted_cave
  Pr

In [68]:
result_df

,Site Name,Max Hazard Score,Med. Oak Suitability,Min Oak Suitability,Max Oak Suitability,Med. Rx Herbivory Suitability,Min Rx Herbivory Suitability,Max Rx Herbivory Suitability,Med. Avo Suitability,Min Avo Suitability,Max Avo Suitability,Med. Citrus Suitability,Min Citrus Suitability,Max Citrus Suitability,Med. Low-Water Suitability,Min Low-Water Suitability,Max Low-Water Suitability,Med. Riparian Suitability,Min Riparian Suitability,Max Riparian Suitability
0,trout_club,4.0,5.0,5.0,5.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,painted_cave,4.0,4.0,4.0,4.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,6.0,6.0,6.0,0.0,0.0,0.0
2,westmont,4.0,7.0,7.0,7.0,8.0,8.0,8.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,7.0,7.0,0.0,0.0,0.0
3,ennisbrook_herb,4.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,0.0,0.0
4,ennisbrook_eucs,4.0,8.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,san_roque,4.0,4.0,4.0,4.0,5.0,5.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0
6,sb_bot_garden,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,elings_bucket,4.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.0,5.0,5.0,7.0,7.0,7.0,0.0,0.0,0.0
8,hollister_p70,4.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,6.0,6.0,5.0,5.0,5.0,6.0,6.0,6.0,0.0,0.0,0.0


In [69]:
# List of columns we want to keep
columns_to_keep = [
    'Site Name',
    'Max Hazard Score',
    'Med. Oak Suitability',
    'Med. Rx Herbivory Suitability',
    'Med. Avo Suitability',
    'Med. Citrus Suitability',
    'Med. Low-Water Suitability',
    'Med. Riparian Suitability'
]

# Select only the desired columns
final_df = result_df[columns_to_keep]

# Optionally, rename the columns to match the exact names requested
column_rename = {
    'Med. Oak Suitability': 'Med. Oak Suitability',
    'Med. Rx Herbivory Suitability': 'Rx Herbivory Suitability',
    'Med. Avo Suitability': 'Avo Suitability',
    'Med. Citrus Suitability': 'Citrus Suitability',
    'Med. Low-Water Suitability': 'Low-Water Suitability',
    'Med. Riparian Suitability': 'Riparian Suitability'
}
final_df = final_df.rename(columns=column_rename)

# Round numeric columns to whole numbers
numeric_columns = final_df.select_dtypes(include=['float64']).columns
#final_df[numeric_columns] = final_df[numeric_columns]
final_df[numeric_columns] = final_df[numeric_columns].round(0)

In [70]:
final_df

,Site Name,Max Hazard Score,Med. Oak Suitability,Rx Herbivory Suitability,Avo Suitability,Citrus Suitability,Low-Water Suitability,Riparian Suitability
0,trout_club,4.0,5.0,3.0,1.0,1.0,1.0,0.0
1,painted_cave,4.0,4.0,5.0,4.0,4.0,6.0,0.0
2,westmont,4.0,7.0,8.0,6.0,6.0,7.0,0.0
3,ennisbrook_herb,4.0,0.0,0.0,8.0,8.0,8.0,0.0
4,ennisbrook_eucs,4.0,8.0,0.0,0.0,0.0,0.0,0.0
5,san_roque,4.0,4.0,5.0,2.0,2.0,2.0,0.0
6,sb_bot_garden,4.0,0.0,0.0,0.0,0.0,0.0,0.0
7,elings_bucket,4.0,6.0,6.0,6.0,5.0,7.0,0.0
8,hollister_p70,4.0,5.0,5.0,6.0,5.0,6.0,0.0


In [ ]:
# Save to CSV
output_csv = os.path.join(raster_folder, 'RWMP_project_summary.csv')
result_df.to_csv(output_csv, index=False)

print(f"Summary table saved to: {output_csv}")

In [31]:
def process_rasters(points, hazard_vector, rasters, buffer_size=100, use_original=False):
    gdf_points = gpd.read_file(points)
    gdf_points['geometry'] = gdf_points.geometry.buffer(buffer_size)
    gdf_hazard = gpd.read_file(hazard_vector)
    
    if gdf_points.crs != gdf_hazard.crs:
        gdf_hazard = gdf_hazard.to_crs(gdf_points.crs)
    
    results = []
    
    for index, row in gdf_points.iterrows():
        site_name = row['name']
        print(f"Processing site: {site_name}")
        
        site_results = {'Site Name': site_name}
        
        hazard_info = gdf_hazard[gdf_hazard.intersects(row['geometry'])]
        if not hazard_info.empty:
            site_results['Max Hazard Score'] = hazard_info['hzds__max'].values[0]
        else:
            site_results['Max Hazard Score'] = "No intersection"
        
        for raster_name, raster_paths in rasters.items():
            raster_path = raster_paths['original'] if use_original else raster_paths['reclassified']
            print(f"  Processing raster: {raster_name}")
            stats = zonal_stats(row['geometry'], raster_path, stats=['median', 'count', 'min', 'max'])
            
            if stats[0]['count'] > 0:
                site_results[f'Med. {raster_name}'] = stats[0]['median']
            else:
                with rasterio.open(raster_path) as src:
                    point_value = list(src.sample([row['geometry'].centroid.coords[0]]))[0][0]
                site_results[f'Med. {raster_name}'] = point_value
        
        results.append(site_results)
    
    return pd.DataFrame(results)

In [32]:
# Paths
project_points = r"C:\Users\bsf31\Documents\post-meds\data\signal\landscape\rwmp_project_sites.gpkg"
hazard_score = r"C:\Users\bsf31\Documents\post-meds\data\signal\landscape\rwmp_watersheds_800ftsplit_firehzdscore_dslpwinds_epsg2229.gpkg"
raster_folder = r"C:\Users\bsf31\Documents\post-meds\data\signal\geoserver\priority"

# Define rasters with both original and reclassified paths
rasters = {
    'Oak Suitability': {
        'original': os.path.join(raster_folder, "suitable_for_oak.tif"),
        'reclassified': os.path.join(raster_folder, "suitable_for_oak_reclassified.tif")
    },
    'Rx Herbivory Suitability': {
        'original': os.path.join(raster_folder, "prescribed_herbivory.tif"),
        'reclassified': os.path.join(raster_folder, "prescribed_herbivory_reclassified.tif")
    },
    'Avo/Citrus Suitability': {
        'original': os.path.join(raster_folder, "avocado_suitability.tif"),
        'reclassified': os.path.join(raster_folder, "avocado_suitability_reclassified.tif")
    },
    'Low-Water Suitability': {
        'original': os.path.join(raster_folder, "low_water_plantation_suitability.tif"),
        'reclassified': os.path.join(raster_folder, "low_water_plantation_suitability_reclassified.tif")
    },
    'Riparian Suitability': {
        'original': os.path.join(raster_folder, "riparian_restoration_priority.tif"),
        'reclassified': os.path.join(raster_folder, "riparian_restoration_priority_reclassified.tif")
    }
}


In [42]:
# Process rasters (set use_original=True for original data, False for reclassified)
result_df = process_rasters(project_points, hazard_score, rasters, buffer_size=100, use_original=False)

# Select and format final columns
columns_to_keep = [
    'Site Name',
    'Max Hazard Score',
    'Med. Oak Suitability',
    'Med. Rx Herbivory Suitability',
    'Med. Avo/Citrus Suitability',
    'Med. Low-Water Suitability',
    'Med. Riparian Suitability'
]

final_df = result_df[columns_to_keep]

column_rename = {
    'Med. Oak Suitability': 'Oak Suitability',
    'Med. Rx Herbivory Suitability': 'Rx Herbivory Suitability',
    'Med. Avo/Citrus Suitability': 'Avo/Citrus Suitability',
    'Med. Low-Water Suitability': 'Low-Water Suitability',
    'Med. Riparian Suitability': 'Riparian Suitability'
}
final_df = final_df.rename(columns=column_rename)

# Round numeric columns to whole numbers
numeric_columns = final_df.select_dtypes(include=['float64']).columns
final_df[numeric_columns] = final_df[numeric_columns].round(0)


Processing site: trout_club
  Processing raster: Oak Suitability
  Processing raster: Rx Herbivory Suitability
  Processing raster: Avo/Citrus Suitability
  Processing raster: Low-Water Suitability
  Processing raster: Riparian Suitability
Processing site: painted_cave
  Processing raster: Oak Suitability
  Processing raster: Rx Herbivory Suitability
  Processing raster: Avo/Citrus Suitability
  Processing raster: Low-Water Suitability
  Processing raster: Riparian Suitability
Processing site: westmont
  Processing raster: Oak Suitability
  Processing raster: Rx Herbivory Suitability
  Processing raster: Avo/Citrus Suitability
  Processing raster: Low-Water Suitability
  Processing raster: Riparian Suitability
Processing site: ennisbrook_herb
  Processing raster: Oak Suitability
  Processing raster: Rx Herbivory Suitability
  Processing raster: Avo/Citrus Suitability
  Processing raster: Low-Water Suitability
  Processing raster: Riparian Suitability
Processing site: ennisbrook_eucs
  

In [43]:
final_df

,Site Name,Max Hazard Score,Oak Suitability,Rx Herbivory Suitability,Avo/Citrus Suitability,Low-Water Suitability,Riparian Suitability
0,trout_club,4.0,5.0,3.0,1.0,1.0,0.0
1,painted_cave,4.0,4.0,5.0,4.0,6.0,0.0
2,westmont,4.0,7.0,8.0,6.0,7.0,0.0
3,ennisbrook_herb,4.0,0.0,0.0,8.0,8.0,0.0
4,ennisbrook_eucs,4.0,8.0,0.0,0.0,0.0,0.0
5,san_roque,4.0,4.0,5.0,2.0,2.0,0.0
6,sb_bot_garden,4.0,0.0,0.0,0.0,0.0,0.0
7,elings_bucket,4.0,6.0,6.0,6.0,7.0,0.0
8,hollister_p70,4.0,5.0,5.0,6.0,6.0,0.0
